In [109]:
!python -m pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 1.4 MB/s eta 0:00:001.6 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 MB 6.0 MB/s eta 0:00:00m eta 0:00:010:00:01


In [110]:
import matplotlib.pyplot as plt

In [133]:
from sortedcontainers import SortedList as ost
from math import floor,ceil,log,sqrt
from collections import deque
import numpy as np
from statistics import mean,variance

In [9]:
class normal_mean :
    def __init__(self,wsize) :
        if wsize != None : wsize += 1
        self.Y = deque([0.0],maxlen = wsize)
        self.SY = deque([0.0],maxlen = wsize)
        self.SYY = deque([0.0],maxlen = wsize)
    def push(self,y) :
        self.Y.append(y)
        self.SY.append(y + self.SY[-1])
        self.SYY.append(y*y + self.SYY[-1])
        # TODO - if Y is full - normalize
    def size(self) :
        return len(self.Y) - 1
    def __call__(self,a,b) :
        val = self.SY[b+1] - self.SY[a]
        val *= val
        val /= (b - a + 1)
        val = -val
        val += self.SYY[b+1] - self.SYY[a]
        return val

In [74]:
def amoc(C,penalty) :
    c,pos = min([(C(0,C.size()-1),0)] + [(C(0,i-1) + C(i,C.size()-1),i) for i in range(1,C.size())])
    return (c + penalty,C.size() - pos - 1) if c + penalty <= C(0,C.size()-1) else  (C(0,C.size()-1),C.size()-1)

In [156]:
Z = list(np.random.normal(0, 1, 1000)) + list(np.random.normal(1, 1, 1000)) + list(np.random.normal(0, 1, 1000))
mu = mean(Z)
sigma = sqrt(variance(Z))
Z = [float((z - mu)/sigma) for z in Z]
n = len(Z)
penalty = 2*log(n)

In [157]:
C = normal_mean(len(Z))
for z in Z :
    C.push(z)

amoc(C,penalty)                                              

(2841.1065618688226, 1998)

In [158]:
class cpts :
    def __init__(self,wsize) :
        self.wsize = wsize
        self.F = deque(maxlen = wsize)
    def push(self,val) :
        self.F.append(val)
        # TODO if F is full - normalize
    def __call__(self,i) :
        return self.F[i]
    def size(self) :
        return len(self.F)
        

In [159]:
def op(C,F,penalty) :
    class C_prime :
        nonlocal C,F
        def __call__(self,i,j) :
            if j == C.size() - 1 :
                return C(i,j)
            return F(j)[0]
        def size(self) :
            return C.size()
    F.push(amoc(C_prime(),penalty))
    return F
        
        

In [164]:
C = normal_mean(n)
F = cpts(n)
for z in Z :
    C.push(z)
    F = op(C,F,penalty)

res = [F(i) for i in range(F.size())]

In [165]:
F(2999)

(2489.609568942829, 995)

In [166]:
F(2999 - 995)

(1688.9196994994413, 1003)

In [167]:
F(2999 - 995 - 1003)

(871.7961542792826, 1001)

In [168]:
F(2999 - 995 - 1003 - 1001)

(0.0, 0)

In [169]:
C(0,2999)

2999.000000000002